In [1]:
import pandas as pd
df = pd.read_json ('drill_report_data_confidential.json')
df.to_csv ('data_excel_confidential.csv', index = None)

df1 = (df.groupby('file_no')['text']
       .apply(lambda x: ','.join(x.dropna().unique()))
       .reset_index())
texty = df1['text'].values.tolist()

In [ ]:
for i in range(len(texty)):
    texty[i] = texty[i].replace("\r", " ").replace("\t", " ").replace("\n"," ").replace('"', "").replace("'", "").replace('/',' ')

### NLTK preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import sys
flag =1
WL = WordNetLemmatizer()

import spacy
nlp = spacy.load('en_core_web_sm')

StopWords = set(stopwords.words('english'))
Filter_number_1 = []

for text2 in texty:
    FilterNumber1 = (re.sub('[^A-Za-z ]+', '', str(text2))).lower() ## To lower and Selecting only alphabets
    filterNumber2 = nlp(FilterNumber1)
    #Tokens = word_tokenize(FilterNumber1)
    Filtered_Sentence = ""
    for word in filterNumber2:
        if word not in StopWords and len(word) > 1:
            Filtered_Sentence = Filtered_Sentence + " " + word.lemma_ #(PS.stem(WL.lemmatize(word)))
    Filter_number_1.append(Filtered_Sentence)
    print(flag)
    flag+=1

### Spell Checker

In [2]:
from symspellpy import SymSpell
import pkg_resources

sym_spell = SymSpell(max_dictionary_edit_distance = 3, prefix_length = 7)
corpus_path = 'Unique_tokens_gloss.txt'
sym_spell.create_dictionary(corpus_path)

dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    print("Dictionary file not found")

In [ ]:
import time
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import sys
Final_Array = []
# lookup suggestions for multi-word input strings (supports compound splitting & merging)
flag = 1725
start = time.time()
for ind_sent in from_1725:  
    if len(ind_sent) > 5000:
        final_ind_row = ''
        #dd_sublen = round(len(ind_sent[0])/10)
        Tokens = word_tokenize(ind_sent)
        dd_sublen = round(len(Tokens)/20)
        for i in range(20):
            dd_sub = ''
            Final_sentt = []
            #dd_sub = ind_sent[0][i*dd_sublen:(i+1)*dd_sublen]
            for j in Tokens[i*dd_sublen:(i+1)*dd_sublen]:
                j+= " "
                dd_sub += j
            suggestions = sym_spell.lookup_compound(dd_sub, max_edit_distance = 2)
            for suggestion in suggestions:
                Final_sentt.append(suggestion)
            final_ind_row += Final_sentt[0].term
        Final_Array.append(final_ind_row)
    else:
        Final_sentt = []
        suggestions = sym_spell.lookup_compound(ind_sent, max_edit_distance = 2)
        for suggestion in suggestions:
            Final_sentt.append(suggestion)
        Final_Array.append(Final_sentt[0].term)
    
    print(flag)
    flag += 1
end = time.time()
print(end - start)

In [ ]:
symspell_dict = [x for x in sym_spell.words.keys()]

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import sys

After_spell_prepro = []

StopWords = set(stopwords.words('english'))
for doc in After_spell:
    if type(doc[0]) is not float:
        Tokens = word_tokenize(doc[0])
        remove_stops = ""
        for word in Tokens:
            if word not in StopWords and len(word) > 1 and word in symspell_dict and word != 'pron':
                remove_stops = remove_stops + " " + word#(PS.stem(WL.lemmatize(word)))
    else:
        remove_stops = ''
    After_spell_prepro.append(remove_stops)

### Noisy Docs removal

In [ ]:
flag=0
flag1=0
short_docs = []
FINAL_LIST = []
After_filter_by_len_files = []
for ind, doc in enumerate(After_spell_prepro):    
    if type(doc[0]) == float or len(doc[0]) <= 100:
        flag+=1
        short_docs.append([file_number[ind],doc[0]])
    elif(len(doc[0]) > 100):
        flag1+=1
        FINAL_LIST.append([file_number[ind],doc[0]])
        
print("Total docs: ", len(After_spell_prepro))        
print("No chars: ", flag)
print("<100 chars: ",flag1)

In [ ]:
from azure.storage.blob import BlobServiceClient

connection_string = "DefaultEndpointsProtocol=https;AccountName=lesslerstore;AccountKey=8lmfbr9xmsPk3ZXhRLGveFoPyLtjJ05xkjOC6UN8jJfLBzivlld3C12awoUYSVdjIOl6Vt5Md2SD6a6CCTH13w==;EndpointSuffix=core.windows.net"
service = BlobServiceClient.from_connection_string(conn_str=connection_string)

from azure.storage.blob import BlobClient
import os
filesss = ['2200','17411']
for file_number in filesss:
    #file_number = '16071'
    file_name_list = []
    for pag in range(1,50):
        try:
            blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="lesslerfilestore/PDForiginal/File_No_" + file_number + "/images", blob_name="File_No_"+file_number+"_PageNo_"+str(pag))
            file_name_list.append("page_"+str(pag)+".jpg") 
            with open(os.path.join(r"Raw_data_original", "page_"+str(pag)+".jpg"), "wb") as my_blob:
                blob_data = blob.download_blob()
                blob_data.readinto(my_blob)
        except:
            print("Check directory for files")
            break

    #------------------------------------------------------------

    from fpdf import FPDF
    pdf = FPDF()
    # imagelist is the list with all image filenames
    for image in file_name_list[:-1]:
        pdf.add_page()
        pdf.image("Raw_data_original/"+image,0,0,210,297)
    pdf.output("Raw_data_original/"+file_number+".pdf", "F")

### N-grams generation

In [ ]:
import gensim
from gensim.corpora import Dictionary
tokenized_text = []
for j in data_:
    tokenized_text.append(j[1].split())

In [ ]:
bigram = gensim.models.Phrases(tokenized_text, min_count = 30) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_text])

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#print(trigram_mod[bigram_mod[tokenized_text[0]]])


bi_tri_grams = []
with_grams = [trigram_mod[bigram_mod[doc]] for doc in tokenized_text]

for doc in with_grams:
     #for token in trigram[bigram[tokenized_text[idx]]]:
    for token in doc:
        
        if '_' in token:
    #             tokenized_text[idx].append(token)
            bi_tri_grams.append(token)
            
# len(bi_tri_grams)

In [ ]:
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt

word_could_dict=Counter(bi_tri_grams)
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(word_could_dict)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()
plt.savefig('yourfile.png', bbox_inches='tight')
plt.close()

In [ ]:
dict(sorted(word_could_dict.items(), key=lambda item: item[1], reverse = True)[:20])

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

ngram_docs = []
for tokens in with_grams:
    ngram_docs.append(TreebankWordDetokenizer().detokenize(tokens))
        
        
df_spell_s = pd.DataFrame(ngram_docs)
df_spell_s.to_csv (r'Ngrams_docs.csv', index = None)